In [14]:
# Installiere benötigte Pakete (einmal ausführen)
#%pip install pandas numpy matplotlib ipython display colorama tensorflow ipywidgets qgrid

In [15]:
# -------------------------------
# Imports
# -------------------------------

# Standard Library
import json
import os
import re
from datetime import datetime

# Third-party packages
import ipywidgets as widgets
import numpy as np
import pandas as pd
from IPython.display import display, clear_output, HTML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Sequential, load_model

import requests, time


In [16]:
# Pfad zur Settings-Datei
settings_file = "settings.json"

# Default Settings (wie besprochen)
default_settings = {
    "allgemein_settings": {
        "model_folder": "models",
        "use_model_file": "",
        "sequence_length": 60,
        "train_epochs": 100,
        "batch_size": 32,
        "validation_split": 0.2,
        "early_stopping_patience": 5,
        "feature_scaling": True,
        "use_dropout": True,
        "dropout_rate": 0.2,
        "loss_function": "binary_crossentropy",
        "optimizer": "adam",
        "debug_mode": True
    },
    "offline": {
        "enabled": True,
        "csv_folder": "csv/binance/",  # Pfad zu deinem CSV Ordner
        "csv_file": "BTCUSDT-1m-6mo-binance-2025-09-12_22-22-26.csv"
    },
    "online": {
        "enabled": False,
        "symbols": ["BTC", "ETH", "SOL"],
        "currencies": ["USDT", "USDT", "USDT"],
        "interval": "1m",
        "poll_seconds": 10,
        "max_live_train_minutes": 60
    },
    "balance": {
        "use_simulation": True,
        "initial_balance": 1000,
        "save_on_profit_percent": 50,
        "game_over_threshold": 0,
        "balance_reward_factor": 1.0,
        "retrain_from_best": True
    },
    "training_options": {
        "use_weighting": True,
        "punish_on_wrong": True,
        "predict_confidence": True,
        "log_transactions": True
    }
}

# Datei erstellen falls nicht existiert
if not os.path.exists(settings_file):
    with open(settings_file, "w") as f:
        json.dump(default_settings, f, indent=4)
    print(f"[INFO] settings.json erstellt")
else:
    print("[INFO] settings.json existiert bereits")

# Laden der Settings
with open(settings_file, "r") as f:
    settings = json.load(f)

print(settings)


[INFO] settings.json existiert bereits
{'allgemein_settings': {'model_folder': 'models', 'use_model_file': '', 'sequence_length': 128, 'train_epochs': 1000, 'batch_size': 32, 'validation_split': 0.2, 'early_stopping_patience': 5, 'feature_scaling': True, 'use_dropout': True, 'dropout_rate': 0.2, 'loss_function': 'binary_crossentropy', 'optimizer': 'adam', 'debug_mode': True}, 'offline': {'enabled': True, 'csv_folder': 'csv/binance', 'csv_file': 'BTCUSDT-1m-1y-binance-2025-09-13_11-37-53.csv'}, 'online': {'symbols': ['BTC', 'ETH', 'SOL'], 'currencies': ['USDT', 'USDT', 'USDT'], 'interval': '1m', 'poll_seconds': 10, 'max_live_train_minutes': 60}, 'balance': {'use_simulation': True, 'initial_balance': 1000, 'save_on_profit_percent': 50, 'game_over_threshold': 0, 'balance_reward_factor': 1.0, 'retrain_from_best': True}, 'training_options': {'use_weighting': True, 'punish_on_wrong': True, 'predict_confidence': True, 'log_transactions': True, 'continual_learning': True, 'transfer_learning': 

In [17]:
# Alle Hauptabschnitte anzeigen
for section, content in settings.items():
    print(f"=== {section.upper()} ===")
    # Wenn es ein Dict ist, in DataFrame umwandeln, sonst direkt anzeigen
    if isinstance(content, dict):
        df = pd.DataFrame(list(content.items()), columns=["Parameter", "Wert"])
        display(df)
    else:
        print(content)
    print("\n")

=== ALLGEMEIN_SETTINGS ===


,Parameter,Wert
0,model_folder,models
1,use_model_file,
2,sequence_length,128
3,train_epochs,1000
4,batch_size,32
5,validation_split,0.2
6,early_stopping_patience,5
7,feature_scaling,True
8,use_dropout,True
9,dropout_rate,0.2




=== OFFLINE ===


,Parameter,Wert
0,enabled,True
1,csv_folder,csv/binance
2,csv_file,BTCUSDT-1m-1y-binance-2025-09-13_11-37-53.csv




=== ONLINE ===


,Parameter,Wert
0,symbols,"[BTC, ETH, SOL]"
1,currencies,"[USDT, USDT, USDT]"
2,interval,1m
3,poll_seconds,10
4,max_live_train_minutes,60




=== BALANCE ===


,Parameter,Wert
0,use_simulation,True
1,initial_balance,1000
2,save_on_profit_percent,50
3,game_over_threshold,0
4,balance_reward_factor,1.0
5,retrain_from_best,True




=== TRAINING_OPTIONS ===


,Parameter,Wert
0,use_weighting,True
1,punish_on_wrong,True
2,predict_confidence,True
3,log_transactions,True
4,continual_learning,True
5,transfer_learning,True
6,ensemble_enabled,True


In [18]:
# Settings laden
settings_file = "settings.json"
with open(settings_file, "r") as f:
    settings = json.load(f)

# Dropdown für die Hauptabschnitte
section_dropdown = widgets.Dropdown(
    options=list(settings.keys()),
    description='Abschnitt:',
    value=list(settings.keys())[0],
    disabled=False,
)

# Output-Bereich für die Tabelle
out = widgets.Output()

# Funktion zum Anzeigen des ausgewählten Abschnitts
def show_section(change):
    with out:
        clear_output()
        section = change['new']
        content = settings[section]
        if isinstance(content, dict):
            df = pd.DataFrame(list(content.items()), columns=["Parameter", "Wert"])
            display(df)
        else:
            print(content)

# Event-Listener für Dropdown
section_dropdown.observe(show_section, names='value')

# Initial anzeigen
show_section({'new': section_dropdown.value})

# Anzeige im Notebook
display(section_dropdown, out)


Dropdown(description='Abschnitt:', options=('allgemein_settings', 'offline', 'online', 'balance', 'training_op…

Output()

In [19]:

# Funktion: Abschnitt als DataFrame anzeigen
def get_section_df(section_name):
    if section_name in settings:
        df = pd.DataFrame(settings[section_name], index=[0]).T
        df.columns = ['Value']
        return df
    else:
        return pd.DataFrame()

# Widgets für die Bearbeitung
section_dropdown = widgets.Dropdown(
    options=list(settings.keys()),
    description='Section:'
)

output = widgets.Output()

# Funktion zum Aktualisieren des DataFrames
def update_df(change):
    section = change['new']
    df = get_section_df(section)
    
    # Erstelle Textboxen für jeden Wert
    textboxes = {}
    for idx, row in df.iterrows():
        for col in df.columns:
            value = row[col]
            tb = widgets.Text(value=str(value), description=idx)
            textboxes[idx] = tb
    
    save_button = widgets.Button(description="Speichern")
    
    # Callback für speichern
    def save_settings(btn):
        for key, tb in textboxes.items():
            val = tb.value
            # Typ erkennen
            if val.lower() == 'true':
                val = True
            elif val.lower() == 'false':
                val = False
            else:
                try:
                    if '.' in val:
                        val = float(val)
                    else:
                        val = int(val)
                except:
                    pass  # bleibt string
            settings[section][key] = val
        
        # Speichern
        with open(settings_file, "w") as f:
            json.dump(settings, f, indent=4)
        with output:
            clear_output()
            print(f"[INFO] Änderungen in '{section}' gespeichert!")
    
    save_button.on_click(save_settings)
    
    with output:
        clear_output()
        for tb in textboxes.values():
            display(tb)
        display(save_button)

# Event handler
section_dropdown.observe(update_df, names='value')

# Anzeige
display(section_dropdown)
display(output)

# Standardanzeige beim Start
update_df({'new': section_dropdown.value})


Dropdown(description='Section:', options=('allgemein_settings', 'offline', 'online', 'balance', 'training_opti…

Output()

In [20]:
# Dropdown für Sektionen
section_names = list(settings.keys())
section_dropdown = widgets.Dropdown(options=section_names, description="Section:")
display(section_dropdown)

# Ausgabebereich
out = widgets.Output()
display(out)

def update_table(section_name):
    out.clear_output()
    df = pd.DataFrame.from_dict(settings[section_name], orient='index', columns=['Value'])
    with out:
        display(df)
    return df

# Initiale Anzeige
current_df = update_table(section_dropdown.value)

# Callback für Dropdown-Wechsel
def on_section_change(change):
    global current_df
    current_df = update_table(change['new'])

section_dropdown.observe(on_section_change, names='value')

# Button zum Speichern
save_button = widgets.Button(description="Speichern")
display(save_button)

def save_clicked(b):
    # Aktualisiere Werte aus DataFrame zurück in Settings
    for key, val in current_df['Value'].items():
        settings[section_dropdown.value][key] = val
    with open(settings_file, "w") as f:
        json.dump(settings, f, indent=4)
    print(f"[INFO] Änderungen in '{section_dropdown.value}' gespeichert!")

save_button.on_click(save_clicked)


Dropdown(description='Section:', options=('allgemein_settings', 'offline', 'online', 'balance', 'training_opti…

Output()

Button(description='Speichern', style=ButtonStyle())

In [ ]:
# Startordner für CSVs (relativer Pfad)
notebook_dir = os.getcwd()
start_folder = os.path.abspath(os.path.join(notebook_dir, f"../../{settings['offline']['csv_folder']}"))

# Widgets
folder_dropdown = widgets.Dropdown(description="Ordner:", layout=widgets.Layout(width="70%"))
file_dropdown = widgets.Dropdown(description="Datei:", layout=widgets.Layout(width="70%"))
save_button = widgets.Button(description="Speichern", button_style="success")
output = widgets.Output()

def update_folder_options(folder):
    """Alle Unterordner anzeigen + aktuelle CSV-Dateien"""
    items = [d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]
    folder_dropdown.options = [".."] + items
    folder_dropdown.value = None
    
    files = [f for f in os.listdir(folder) if f.endswith(".csv")]
    file_dropdown.options = files
    if settings['offline'].get('csv_file') in files:
        file_dropdown.value = settings['offline']['csv_file']
    elif files:
        file_dropdown.value = files[0]

# Callback bei Ordnerwechsel
def folder_changed(change):
    if change['new'] is not None:
        selected = change['new']
        if selected == "..":
            new_folder = os.path.abspath(os.path.join(current_folder[0], ".."))
        else:
            new_folder = os.path.join(current_folder[0], selected)
        if os.path.exists(new_folder):
            current_folder[0] = new_folder
            update_folder_options(new_folder)
            with output:
                clear_output()
                print(f"[INFO] Aktueller Ordner: {new_folder}")

# Callback zum Speichern
def save_csv(b):
    selected_file = file_dropdown.value
    settings['offline']['csv_file'] = selected_file
    with open(settings_file, "w") as f:
        json.dump(settings, f, indent=4)
    with output:
        clear_output()
        print(f"[INFO] CSV-Datei in settings.json auf '{selected_file}' gesetzt")

# Aktueller Ordner als mutable Variable
current_folder = [start_folder]
update_folder_options(start_folder)

folder_dropdown.observe(folder_changed, names='value')
save_button.on_click(save_csv)

display(folder_dropdown, file_dropdown, save_button, output)


Dropdown(description='Ordner:', layout=Layout(width='70%'), options=('..',), value=None)

Dropdown(description='Datei:', index=10, layout=Layout(width='70%'), options=('BTCUSDT-1m-3mo-binance-2025-09-…

Button(button_style='success', description='Speichern', style=ButtonStyle())

Output()

In [22]:
csv_folder = os.path.abspath(os.path.join(os.getcwd(), f"../../{settings['offline']['csv_folder']}"))
csv_file = settings['offline'].get('csv_file')
csv_path = os.path.join(csv_folder, csv_file)

if not os.path.exists(csv_path):
    raise FileNotFoundError(f"[ERROR] CSV-Datei existiert nicht: {csv_path}")

print(os.path.exists(csv_path))
# -------------------------------
# CSV laden
# -------------------------------
df = pd.read_csv(csv_path, parse_dates=True)

# -------------------------------
# Pagination Setup
# -------------------------------
page_size = 6
num_pages = (len(df) + page_size - 1) // page_size
current_page = 1

# Widgets
btn_prev = widgets.Button(description="◀ Back", button_style='')
btn_next = widgets.Button(description="Next ▶", button_style='')
page_label = widgets.Label(value=f"Page {current_page} of {num_pages}")
output = widgets.Output()

# Header einmal anzeigen
display(HTML(df.head(0).to_html(index=False)))

def show_page(page):
    start = (page - 1) * page_size
    end = start + page_size
    with output:
        clear_output()
        display(HTML(df.iloc[start:end].to_html(header=False, index=False)))
    page_label.value = f"Page {page} of {num_pages}"

def on_prev(b):
    global current_page
    if current_page > 1:
        current_page -= 1
        show_page(current_page)

def on_next(b):
    global current_page
    if current_page < num_pages:
        current_page += 1
        show_page(current_page)

btn_prev.on_click(on_prev)
btn_next.on_click(on_next)

# Initialanzeige
show_page(current_page)

# Anzeige
display(widgets.HBox([btn_prev, btn_next, page_label]))
display(output)


True


timestamp,symbol,open,high,low,close,prev_close,current_close,volume,color


Output()

In [23]:
# -------------------------------
# Settings laden
# -------------------------------
with open("settings.json", "r") as f:
    settings = json.load(f)

# Allgemein
allg = settings['allgemein_settings']
offline = settings['offline']
balance_settings = settings['balance']
train_opts = settings['training_options']


In [24]:
# -------------------------------
# CSV laden (offline)
# -------------------------------

df = pd.read_csv(csv_path, parse_dates=['timestamp'])

# Label: Grün=1, Rot=0
df['label'] = (df['color']=='green').astype(int)

features = ['open','high','low','close','prev_close','current_close','volume']
X = df[features].values
y = df['label'].values

# Feature Scaling
if allg.get('feature_scaling', True):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=allg.get('validation_split',0.2), shuffle=False)


In [25]:
# -------------------------------
# Modell Handling
# -------------------------------
model_folder = allg.get('model_folder','models')
os.makedirs(model_folder, exist_ok=True)

model_file = allg.get('use_model_file')
if model_file and os.path.exists(os.path.join(model_folder, model_file)):
    model = load_model(os.path.join(model_folder, model_file))
    print(f"Modell geladen: {model_file}")
else:
    # Neues Modell erstellen
    model = Sequential([
        Input(shape=(X_train.shape[1],)),  # X_train.shape[1] = 7 jetzt mit Volume
        Dense(64, activation='relu'),
        Dropout(allg.get('dropout_rate',0.2)) if allg.get('use_dropout', True) else tf.keras.layers.Lambda(lambda x:x),
        Dense(32, activation='relu'),
        Dropout(allg.get('dropout_rate',0.2)) if allg.get('use_dropout', True) else tf.keras.layers.Lambda(lambda x:x),
        Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=allg.get('optimizer','adam'),
        loss=allg.get('loss_function','binary_crossentropy'),
        metrics=['accuracy']
    )
    print("Neues Modell erstellt")


Neues Modell erstellt


In [26]:
# -------------------------------
# EarlyStopping
# -------------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=allg.get('early_stopping_patience',5), restore_best_weights=True)


In [27]:
# -------------------------------
# Training
# -------------------------------
history = model.fit(
    X_train, y_train,
    epochs=allg.get('train_epochs',10),
    batch_size=allg.get('batch_size',32),
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/1000
10500/10500 ━━━━━━━━━━━━━━━━━━━━ 9s 789us/step - accuracy: 0.5106 - loss: 0.6931 - val_accuracy: 0.5217 - val_loss: 0.6918
Epoch 2/1000
10500/10500 ━━━━━━━━━━━━━━━━━━━━ 9s 812us/step - accuracy: 0.5178 - loss: 0.6922 - val_accuracy: 0.5614 - val_loss: 0.6878
Epoch 3/1000
10500/10500 ━━━━━━━━━━━━━━━━━━━━ 8s 804us/step - accuracy: 0.6634 - loss: 0.5936 - val_accuracy: 0.8838 - val_loss: 0.3881
Epoch 4/1000
10500/10500 ━━━━━━━━━━━━━━━━━━━━ 8s 804us/step - accuracy: 0.7924 - loss: 0.4136 - val_accuracy: 0.9076 - val_loss: 0.2974
Epoch 5/1000
10500/10500 ━━━━━━━━━━━━━━━━━━━━ 8s 727us/step - accuracy: 0.8012 - loss: 0.3936 - val_accuracy: 0.7472 - val_loss: 0.4090
Epoch 6/1000
10500/10500 ━━━━━━━━━━━━━━━━━━━━ 7s 704us/step - accuracy: 0.8011 - loss: 0.3916 - val_accuracy: 0.9448 - val_loss: 0.2456
Epoch 7/1000
10500/10500 ━━━━━━━━━━━━━━━━━━━━ 8s 786us/step - accuracy: 0.8034 - loss: 0.3877 - val_accuracy: 0.8986 - val_loss: 0.3206
Epoch 8/1000
10500/10500 ━━━━━━━━━━━━━━━━━━━━ 8s

In [ ]:
# -------------------------------
# Vorhersagen & Balance-Simulation
# -------------------------------
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob>0.5).astype(int)

balance = balance_settings.get('initial_balance',1000)
position = 0
transactions = []

for i in range(len(y_pred)):
    price = df.iloc[len(X_train)+i]['close']
    confidence = float(y_pred_prob[i])
    pred_label = 'Grün' if y_pred[i]==1 else 'Rot'
    actual_label = 'Grün' if y_test[i]==1 else 'Rot'
    profit = 0

    if pred_label=='Grün' and position==0:  # Kauf
        position = price
    elif pred_label=='Rot' and position!=0:  # Verkauf
        profit = (price - position) * balance_settings.get('balance_reward_factor',1.0)
        balance += profit
        position = 0

    transactions.append({
        "timestamp": str(df.iloc[len(X_train)+i]['timestamp']),
        "predicted": pred_label,
        "actual": actual_label,
        "profit": float(profit),
        "confidence": {"Rot": 1-confidence, "Grün": confidence}
    })

# Offene Position am Ende verkaufen
if position!=0:
    balance += df.iloc[-1]['close'] - position

accuracy = (y_pred.flatten() == y_test).mean()
expected_profit = sum([t['profit'] for t in transactions])

print(f"Trefferquote: {accuracy*100:.2f}%")
print(f"Erwarteter Profit: {expected_profit:.2f} USDT")
print(f"Endbalance: {balance:.2f} USDT")


In [ ]:
# -------------------------------
# Transaktionslog speichern
# -------------------------------
balance_settings = settings.get('balance', {'initial_balance':1000}) # Beispiel, anpassen

# Prüfe vorhandene Modelle für Versionierung
existing = [f for f in os.listdir(model_folder) if f.startswith("Heusc_v") and f.endswith(".keras")]

if existing:
    versions = [float(re.search(r"Heusc_v(\d+\.\d+)", f).group(1)) for f in existing]
    next_version = max(versions) + 0.1
else:
    next_version = 0.1

# Modellname: Version + Datum/Zeit
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model_name = f"Heusc_v{next_version:.1f}_{timestamp}.keras"

#model_name = f"model_{datetime.now().strftime('%Y%m%d_%H%M%S')}.keras"

model_status = "success" if balance>balance_settings.get('initial_balance',1000) else "game_over"

model.save(os.path.join(model_folder, model_name))

log_data = {
    "model": model_name,
    "created": str(datetime.now()),
    "status": model_status,
    "balance": balance,
    "transactions": transactions
}

log_file = os.path.join(model_folder, model_name.replace('.keras','.json'))
with open(log_file,'w') as f:
    json.dump(log_data,f,indent=4)

print(f"Modell und Log gespeichert: {model_name}")

In [ ]:
# -------------------------------
# Modell Auswahl + Retrain mit Bestätigen (CSV schon vorher definiert)
# -------------------------------

model_folder = os.path.abspath(allg.get('model_folder', 'models'))
use_model_file = allg.get('use_model_file', '')

# Widgets
model_dropdown = widgets.Dropdown(description="Modell:", layout=widgets.Layout(width="70%"))
confirm_button = widgets.Button(description="Bestätigen", button_style="success")
retrain_button = widgets.Button(description="Retrain", button_style="info")
output = widgets.Output()

last_confirmed_model = None

# -------------------------------
# Optionen aktualisieren
# -------------------------------
def update_model_options():
    files = [f for f in os.listdir(model_folder) if f.endswith(".keras")]
    model_dropdown.options = files
    if use_model_file in files:
        model_dropdown.value = use_model_file
    elif files:
        model_dropdown.value = files[0] if files else None

# -------------------------------
# Bestätigen Button
# -------------------------------
def confirm_selection(b):
    global last_confirmed_model
    selected_model = model_dropdown.value
    selected_csv = settings['offline'].get('csv_file')

    if selected_model and selected_model != last_confirmed_model:
        last_confirmed_model = selected_model
        allg['use_model_file'] = selected_model
        
        # JSON-Log des Modells prüfen
        log_file = os.path.join(model_folder, selected_model.replace('.keras','.json'))
        if os.path.exists(log_file):
            with open(log_file,'r') as f:
                log_data = json.load(f)
            balance = log_data.get('balance', balance_settings.get('initial_balance',1000))
            transactions = log_data.get('transactions', [])
        else:
            balance = balance_settings.get('initial_balance',1000)
            transactions = []

        with output:
            clear_output()
            print(f"[INFO] Modell bestätigt: {selected_model}")
            print(f"[INFO] CSV verwendet: {selected_csv}")
            print(f"[INFO] {len(transactions)} Transaktionen aus JSON geladen, Balance={balance}")

        # Settings speichern
        with open(settings_file, "w") as f:
            json.dump(settings, f, indent=4)
    else:
        with output:
            clear_output()
            print(f"[INFO] Modell bereits bestätigt: {selected_model}")

# -------------------------------
# Retrain Funktion
# -------------------------------
def retrain_model(b):
    selected_model = allg.get('use_model_file')
    selected_csv = settings['offline'].get('csv_file')

    if not selected_model or not selected_csv:
        with output:
            clear_output()
            print("[WARN] Bitte zuerst Modell und CSV bestätigen!")
        return

    model_path = os.path.join(model_folder, selected_model)
    model = load_model(model_path)
    
    csv_path = os.path.join(start_folder, selected_csv)
    df = pd.read_csv(csv_path, parse_dates=['timestamp'])
    
    # Hier dein Retraining einfügen (z.B. Prozent-Basis)
    with output:
        clear_output()
        print(f"[INFO] Retraining gestartet für: {selected_model} mit CSV: {selected_csv}")
        # train_model(model, df, transactions)  # <- deine Trainingsfunktion
        print(f"[INFO] Retraining abgeschlossen für: {selected_model}")

# -------------------------------
# Setup
# -------------------------------
update_model_options()

confirm_button.on_click(confirm_selection)
retrain_button.on_click(retrain_model)

display(model_dropdown, confirm_button, retrain_button, output)


In [ ]:
# -------------------------------
# Retrain & Trade-Simulation mit CSV-Auswahl
# -------------------------------

# -------------------------------
# Settings
# -------------------------------
csv_folder = os.path.abspath(os.path.join(os.getcwd(), f"../../{settings['offline']['csv_folder']}"))
balance_settings = settings.get('balance', {'initial_balance': 1000})
risk_per_trade = 0.02  # 2% pro Trade

# -------------------------------
# Widgets
# -------------------------------
model_dropdown = widgets.Dropdown(description="Modell:", layout=widgets.Layout(width="70%"))
csv_dropdown = widgets.Dropdown(description="CSV:", layout=widgets.Layout(width="70%"))
confirm_button = widgets.Button(description="Bestätigen", button_style="success")
retrain_button = widgets.Button(description="Retrain", button_style="info")
output = widgets.Output()

last_confirmed_model = None
last_confirmed_csv = None

# -------------------------------
# Update Dropdowns
# -------------------------------
def update_model_csv_options():
    # Modelle
    model_files = [f for f in os.listdir(model_folder) if f.endswith(".keras")]
    model_dropdown.options = model_files
    if allg.get('use_model_file') in model_files:
        model_dropdown.value = allg.get('use_model_file')
    elif model_files:
        model_dropdown.value = model_files[0]
    
    # CSVs
    csv_files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]
    csv_dropdown.options = csv_files
    if settings['offline'].get('csv_file') in csv_files:
        csv_dropdown.value = settings['offline'].get('csv_file')
    elif csv_files:
        csv_dropdown.value = csv_files[0]

update_model_csv_options()

# -------------------------------
# Bestätigen
# -------------------------------
def confirm_selection(b):
    global last_confirmed_model, last_confirmed_csv
    sel_model = model_dropdown.value
    sel_csv = csv_dropdown.value
    confirmed = []
    
    if sel_model != last_confirmed_model:
        allg['use_model_file'] = sel_model
        last_confirmed_model = sel_model
        confirmed.append(f"Modell bestätigt: {sel_model}")
    
    if sel_csv != last_confirmed_csv:
        settings['offline']['csv_file'] = sel_csv
        last_confirmed_csv = sel_csv
        confirmed.append(f"CSV bestätigt: {sel_csv}")
    
    if confirmed:
        with open(settings_file, "w") as f:
            json.dump(settings, f, indent=4)
        with output:
            clear_output()
            for msg in confirmed:
                print(f"[INFO] {msg}")
    else:
        with output:
            clear_output()
            print("[INFO] Bereits bestätigt")

# -------------------------------
# Retrain
# -------------------------------
def retrain_model(b):
    sel_model = allg.get('use_model_file')
    sel_csv = settings['offline'].get('csv_file')
    
    if not sel_model or not sel_csv:
        with output:
            clear_output()
            print("[WARN] Bitte zuerst Modell und CSV bestätigen!")
        return
    
    # Modell laden
    model_path = os.path.join(model_folder, sel_model)
    model = load_model(model_path)
    
    # CSV laden
    csv_path = os.path.join(csv_folder, sel_csv)
    df = pd.read_csv(csv_path, parse_dates=["timestamp"])
    
    # Features & Labels
    features = ['open','high','low','close','prev_close','current_close','volume']
    X = df[features].values
    y = (df['color'] == 'green').astype(int).values
    
    # Split Train/Test
    split_idx = int(len(X)*0.8)
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    
    # Vorhersage
    y_pred_prob = model.predict(X_test).flatten()
    y_pred = (y_pred_prob > 0.5).astype(int)
    
    # Alte Transaktionen & Balance laden
    log_file = os.path.join(model_folder, sel_model.replace('.keras','.json'))
    try:
        with open(log_file) as f:
            old_log = json.load(f)
            transactions = old_log.get('transactions', [])
            balance = old_log.get('balance', balance_settings.get('initial_balance',1000))
    except:
        transactions = []
        balance = balance_settings.get('initial_balance', 1000)
    
    position = 0
    invested = 0
    
    for i in range(len(y_pred)):
        price = X_test[i,5]  # current_close
        pred_label = 'Grün' if y_pred[i]==1 else 'Rot'
        actual_label = 'Grün' if y_test[i]==1 else 'Rot'
        
        trade_amount = balance * risk_per_trade
        profit = 0

        if pred_label=='Grün' and position==0:
            position = price
            invested = trade_amount
        elif pred_label=='Rot' and position!=0:
            profit = (price - position)/position * invested
            balance += profit
            position = 0
        
        transactions.append({
            "timestamp": str(df.iloc[split_idx+i]['timestamp']),
            "predicted": pred_label,
            "actual": actual_label,
            "profit_cash": float(profit),
            "profit_percent": float(profit/invested*100) if invested>0 else 0,
            "confidence": {"Rot": float(1-y_pred_prob[i]), "Grün": float(y_pred_prob[i])}
        })
    
    if position != 0:
        profit = (X_test[-1,5]-position)/position * invested
        balance += profit
    
    accuracy = (y_pred == y_test).mean()
    expected_profit = sum([t.get('profit_cash',0) for t in transactions])  # KeyError fix
    
    with output:
        clear_output()
        print(f"[INFO] Retraining abgeschlossen für: {sel_model}")
        print(f"Trefferquote: {accuracy*100:.2f}%")
        print(f"Erwarteter Profit: {expected_profit:.2f} USDT")
        print(f"Endbalance: {balance:.2f} USDT")
    
    # Modellversionierung + speichern
    existing = [f for f in os.listdir(model_folder) if f.startswith("Heusc_v") and f.endswith(".keras")]
    next_version = (max([float(re.search(r"Heusc_v(\d+\.\d+)", f).group(1)) for f in existing]) + 0.1) if existing else 0.1
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    model_name = f"Heusc_v{next_version:.1f}_{timestamp}.keras"
    model_status = "success" if balance > balance_settings.get('initial_balance',1000) else "game_over"
    
    model.save(os.path.join(model_folder, model_name))
    
    log_data = {
        "model": model_name,
        "created": str(datetime.now()),
        "status": model_status,
        "balance": balance,
        "transactions": transactions
    }
    
    log_file = os.path.join(model_folder, model_name.replace('.keras','.json'))
    with open(log_file,'w') as f:
        json.dump(log_data, f, indent=4)
    
    with output:
        print(f"[INFO] Modell und Log gespeichert: {model_name}")

# -------------------------------
# Callbacks
# -------------------------------
confirm_button.on_click(confirm_selection)
retrain_button.on_click(retrain_model)

display(model_dropdown, csv_dropdown, confirm_button, retrain_button, output)


In [ ]:
def retrain_until_game_over(model, X_test, y_test, y_pred_prob, risk_per_trade=0.02, initial_balance=1000):
    balance = initial_balance
    transactions = []
    position = 0
    invested = 0

    step = 0
    while balance > 0:  # Einfaches Kriterium: Spiel läuft bis Balance <= 0
        for i in range(len(y_pred_prob)):
            price = X_test[i,5]  # current_close
            pred_label = 'Grün' if y_pred_prob[i] > 0.5 else 'Rot'
            actual_label = 'Grün' if y_test[i]==1 else 'Rot'

            trade_amount = balance * risk_per_trade
            profit = 0

            if pred_label=='Grün' and position==0:
                position = price
                invested = trade_amount
            elif pred_label=='Rot' and position!=0:
                profit = (price - position)/position * invested
                balance += profit
                position = 0

            transactions.append({
                "step": step,
                "predicted": pred_label,
                "actual": actual_label,
                "profit_cash": float(profit),
                "profit_percent": float(profit/invested*100) if invested>0 else 0,
            })

        # Offene Position am Ende verkaufen
        if position != 0:
            profit = (X_test[-1,5]-position)/position * invested
            balance += profit
            position = 0

        step += 1
        print(f"[INFO] Schritt {step}: Balance = {balance:.2f} USDT")

        # Stopp wenn Balance unter Start fällt
        if balance <= initial_balance:
            print("[INFO] Game Over erreicht. Jetzt speichern...")
            # Hier könntest du Modell & Log speichern
            break

# Beispiel-Aufruf:
# retrain_until_game_over(model, X_test, y_test, y_pred_prob)


In [ ]:
def retrain_model(b):
    sel_model = allg.get('use_model_file')
    sel_csv = settings['offline'].get('csv_file')
    
    if not sel_model or not sel_csv:
        with output:
            clear_output()
            print("[WARN] Bitte zuerst Modell und CSV bestätigen!")
        return
    
    # Modell laden
    model_path = os.path.join(model_folder, sel_model)
    model = load_model(model_path)
    
    # CSV laden
    csv_path = os.path.join(csv_folder, sel_csv)
    df = pd.read_csv(csv_path, parse_dates=["timestamp"])
    
    # Features
    features = ['open','high','low','close','prev_close','current_close','volume']
    X = df[features].values
    y = (df['color'] == 'green').astype(int).values
    
    # Split für Simulation (letzte 20%)
    split_idx = int(len(X)*0.8)
    X_test = X[split_idx:]
    y_test = y[split_idx:]
    
    # Vorhersage
    y_pred_prob = model.predict(X_test).flatten()
    
    # Alte Transaktionen laden
    log_file = os.path.join(model_folder, sel_model.replace('.keras','.json'))
    try:
        with open(log_file) as f:
            old_log = json.load(f)
            transactions = old_log.get('transactions', [])
    except:
        transactions = []

    balance = balance_settings.get('initial_balance', 1000)
    position = 0
    invested = 0
    step = 0
    
    with output:
        clear_output()
        print(f"[INFO] Retraining gestartet für: {sel_model}")
    
    # Simulieren bis Game Over
    while balance > balance_settings.get('initial_balance', 1000):
        for i in range(len(y_pred_prob)):
            price = X_test[i,5]  # current_close
            pred_label = 'Grün' if y_pred_prob[i] > 0.5 else 'Rot'
            actual_label = 'Grün' if y_test[i]==1 else 'Rot'
            
            trade_amount = balance * risk_per_trade
            profit = 0

            # Kauf
            if pred_label=='Grün' and position==0:
                position = price
                invested = trade_amount
            # Verkauf
            elif pred_label=='Rot' and position!=0:
                profit = (price - position)/position * invested
                balance += profit
                position = 0
            
            transactions.append({
                "step": step,
                "predicted": pred_label,
                "actual": actual_label,
                "profit_cash": float(profit),
                "profit_percent": float(profit/invested*100) if invested>0 else 0
            })
        
        # Offene Position am Ende verkaufen
        if position != 0:
            profit = (X_test[-1,5]-position)/position * invested
            balance += profit
            position = 0
        
        step += 1
        with output:
            print(f"[INFO] Schritt {step}: Balance = {balance:.2f} USDT")
        
        if balance <= balance_settings.get('initial_balance', 1000):
            with output:
                print("[INFO] Game Over erreicht. Jetzt speichern...")
            break
    
    # Trefferquote & Gesamtprofit
    y_pred = (y_pred_prob > 0.5).astype(int)
    accuracy = (y_pred == y_test).mean()
    expected_profit = sum([t['profit_cash'] for t in transactions])
    
    # Modellversionierung + speichern
    existing = [f for f in os.listdir(model_folder) if f.startswith("Heusc_v") and f.endswith(".keras")]
    next_version = (max([float(re.search(r"Heusc_v(\d+\.\d+)", f).group(1)) for f in existing]) + 0.1) if existing else 0.1
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    model_name = f"Heusc_v{next_version:.1f}_{timestamp}.keras"
    model_status = "success" if balance > balance_settings.get('initial_balance',1000) else "game_over"
    
    model.save(os.path.join(model_folder, model_name))
    
    log_data = {
        "model": model_name,
        "created": str(datetime.now()),
        "status": model_status,
        "balance": balance,
        "transactions": transactions
    }
    
    log_file = os.path.join(model_folder, model_name.replace('.keras','.json'))
    with open(log_file,'w') as f:
        json.dump(log_data, f, indent=4)
    
    with output:
        print(f"[INFO] Retraining abgeschlossen für: {sel_model}")
        print(f"Trefferquote: {accuracy*100:.2f}%")
        print(f"Erwarteter Profit: {expected_profit:.2f} USDT")
        print(f"Endbalance: {balance:.2f} USDT")
        print(f"[INFO] Modell und Log gespeichert: {model_name}")


In [ ]:
# Vorhersage ist fertig
y_pred_prob = model.predict(X_test).flatten()

# Simulation bis Game Over
retrain_until_game_over(model, X_test, y_test, y_pred_prob, 
                        risk_per_trade=risk_per_trade, 
                        initial_balance=balance_settings.get('initial_balance',1000))


In [ ]:
API_URL = "http://127.0.0.1:5000/api/train_mode" #http://127.0.0.1:5000/api/train_mode?symbols=BTCUSDT&source=binance&interval=1m&period=3d
SYMBOL = "BTCUSDT"
SOURCE = "binance"
INTERVAL = "1m"
PERIOD = "3d"

initial_balance = 1000
balance = initial_balance
risk_per_trade = 0.01  # 1% Risiko

In [ ]:
# --- Settings ---
API_URL = "http://127.0.0.1:5000/api/train_mode"
symbol = "BTCUSDT"
source = "binance"
interval = "1m"
lookback_volume = 10  # Candles für Durchschnittsvolume
initial_balance = 1000
risk_per_trade = 0.05  # 1% per Trade

# --- Fetch Candles ---
resp = requests.get(f"{API_URL}?symbols={symbol}&source={source}&interval={interval}")
data = resp.json()

candles = data[symbol]['history'] + [data[symbol]['live']]  # Historische + Live-Candle

# --- Simulation ---
balance = initial_balance
trade_log = []

# Berechne durchschnittliches Volume der letzten N Candles
for i, c in enumerate(candles):
    if i < lookback_volume:
        avg_volume = sum([candles[j]['volume'] for j in range(i+1)]) / (i+1)
    else:
        avg_volume = sum([candles[j]['volume'] for j in range(i-lookback_volume, i)]) / lookback_volume

    # Trade-Bedingung (z. B. Buy, wenn grün und Volume > Durchschnitt)
    take_trade = False
    if c['color'] == 'green' and c['volume'] > avg_volume:
        take_trade = True
        trade_size = balance * risk_per_trade * (c['volume']/avg_volume)  # proportional zum Volumen
        profit = (c['close'] - c['open']) / c['open'] * trade_size
        balance += profit
        trade_log.append({
            "timestamp": c['timestamp'],
            "action": "BUY",
            "open": c['open'],
            "close": c['close'],
            "volume": c['volume'],
            "avg_volume": avg_volume,
            "profit": profit,
            "balance": balance
        })
    else:
        trade_log.append({
            "timestamp": c['timestamp'],
            "action": "HOLD",
            "open": c['open'],
            "close": c['close'],
            "volume": c['volume'],
            "avg_volume": avg_volume,
            "profit": 0,
            "balance": balance
        })

# --- Ausgabe ---
df = pd.DataFrame(trade_log)
print(df.tail(10))
print(f"Start Balance: {initial_balance}, End Balance: {balance:.2f}")

In [ ]:
import os, json, time, requests, numpy as np, pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from datetime import datetime

# -------------------------------
# Settings laden
# -------------------------------
with open("settings.json", "r") as f:
    settings = json.load(f)

allg = settings['allgemein_settings']
online = settings['online']
balance_settings = settings['balance']

# -------------------------------
# Modell laden
# -------------------------------
model_path = os.path.join(allg['model_folder'], allg['use_model_file'])
model = load_model(model_path)
print(f"Model geladen: {model_path}")

scaler = StandardScaler()  # Optional, falls du später Feature Scaling einbauen willst
print(model.summary())
print(scaler)
# -------------------------------
# Online Trading Loop (Live + Train Mode)
# -------------------------------
if not online['enabled']:
    print("Online Mode ist deaktiviert.")
else:
    balance = balance_settings.get('initial_balance', 1000)
    position = 0
    trade_log = []

    sequence_length = 3  # 1 bis 3 Candles für Input
    end_time = pd.Timestamp.now() + pd.Timedelta(minutes=online['max_live_train_minutes'])

    seq_buffer = []  # Sequenz für LSTM

    while pd.Timestamp.now() < end_time and balance > balance_settings.get('game_over_threshold', 0):
        for sym, cur in zip(online['symbols'], online['currencies']):
            url = f"http://127.0.0.1:5000/api/train_mode?symbols={sym}{cur}&source=binance&interval={online['interval']}&period=3d"
            try:
                resp = requests.get(url)
                data = resp.json()
            except Exception as e:
                print(f"[Fehler API] {e}")
                continue

            candles = data.get(f"{sym}{cur}", {}).get("history", []) + [data.get(f"{sym}{cur}", {}).get("live")]

            for c in candles:
                if c is None:
                    continue

                # Sequenz füllen
                seq_buffer.append([c['open'], c['high'], c['low'], c['close'], c['volume']])
                if len(seq_buffer) > sequence_length:
                    seq_buffer.pop(0)

                if len(seq_buffer) == sequence_length:
                    X_seq = np.array(seq_buffer).reshape(1, sequence_length, 5)

                    # Vorhersage
                    y_pred_prob = model.predict(X_seq, verbose=0).flatten()[0]
                    pred_label = "Grün" if y_pred_prob > 0.5 else "Rot"
                    profit = 0

                    # Trade Logik
                    if pred_label == "Grün" and position == 0:
                        position = c['close']
                    elif pred_label == "Rot" and position != 0:
                        profit = (c['close'] - position) * balance_settings.get('balance_reward_factor', 1.0)
                        balance += profit
                        position = 0

                    trade_log.append({
                        "timestamp": c['timestamp'],
                        "symbol": f"{sym}{cur}",
                        "predicted": pred_label,
                        "pred_conf": float(y_pred_prob),
                        "profit": profit,
                        "balance": balance
                    })

        time.sleep(online['poll_seconds'])

    # Offene Position am Ende schließen
    if position != 0:
        balance += candles[-1]['close'] - position

    # -------------------------------
    # Model & Log speichern
    # -------------------------------
    model_name = f"{allg.get('use_model_file').replace('.keras','')}_online_{datetime.now().strftime('%Y%m%d_%H%M%S')}.keras"
    model.save(os.path.join(allg['model_folder'], model_name))

    log_file = os.path.join(allg['model_folder'], model_name.replace('.keras','.json'))
    with open(log_file, 'w') as f:
        json.dump({"model": model_name, "balance": balance, "trades": trade_log}, f, indent=4)

    print(f"Endbalance: {balance:.2f} USDT")
    print(f"Model und Trade-Log gespeichert: {model_name}")
